In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from bart_playground import *

In [3]:
import numpy as np

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
X = abalone.data.features 
y = abalone.data.targets 
X = X.drop(columns=['Sex'])

X = X.values.astype(float)
y = np.array(y).reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [4]:
proposal_probs_mtmh = {
    'multi_grow': 0.5,
    'multi_prune': 0.5
}
bart = MultiBART(ndpost=200, nskip=100, n_trees=100, multi_tries=1, proposal_probs=proposal_probs_mtmh)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:12<00:00, 24.41it/s]


In [5]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success, acceptance and change rates
df["success_rate"] = df["success"] / df["selected"]
df["accept_rate"] = df["accepted"] / df["success"] # The "acceptance rate" reflects the MH acceptance probability
df["change_rate"] = df["accepted"] / df["selected"] # The "change rate" reflects the frequency of tree changes

print(df)

             selected  success  accepted  success_rate  accept_rate  \
multi_grow      14843    14843      4275       1.00000     0.288015   
multi_prune     15157    14245      4109       0.93983     0.288452   

             change_rate  
multi_grow      0.288015  
multi_prune     0.271096  


In [6]:
proposal_probs = {"grow" : 0.5,
                  "prune" : 0.5}

bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:06<00:00, 47.19it/s]


In [7]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success, acceptance and change rates
df["success_rate"] = df["success"] / df["selected"]
df["accept_rate"] = df["accepted"] / df["success"] # The "acceptance rate" reflects the MH acceptance probability
df["change_rate"] = df["accepted"] / df["selected"] # The "change rate" reflects the frequency of tree changes

print(df)

       selected  success  accepted  success_rate  accept_rate  change_rate
grow      15010    15010      4242      1.000000     0.282612     0.282612
prune     14990    14192      4061      0.946765     0.286147     0.270914
